# DNN
1. 把all_keys.csv, ESun_public_y_answer.csv加到工作區
2. 調整 # Main 下方的兩個block，第一個調參數，第二個建network
3. 其他block應該不需要動
4. 最後一個block會印成績

In [402]:
import numpy as np
import pandas as pd
import csv
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
import math

# Read data

In [403]:
!gdown 1vHDhZSPmhithLVRRzkNw0ak_kk7PhInu # V_trade
!gdown 145T8z3XXlsaISzWdrJgJVANGokG1XuFI # V_remit
!gdown 1AubrUmeNUgpgiOu4tay6Gwl8O3lBaokF # V_info
!gdown 1zZo9RLt3mMmJZxEETSY2g9ND31qkZIn0 # V_cred
!gdown 1uFCx21bqE3FnrdfvN_mwtw2-nvogTEex # V_cons
!gdown 1ZOXGT_rIdEGIliHGKEH3ha77ZlZyq1Gn # train_y
!gdown 1qjEwmi97OWdshSNdgQj2ccXnoM4UvT25 # V_trade_public
!gdown 1g8trBiC6OxuoTU94u_UMygVrA-fSASpB # V_remit_public
!gdown 14KTfY56Mz2xBXdP27GvGB2HVeb_4Ks4T # V_info_public
!gdown 1EaIWnjQxUl4KRgVCqYT7AB4PaSNvc_GL # V_cred_public
!gdown 1owf1urxHZywAxJfCgVXCpXEMQ6VZGhnO # V_cons_public
!gdown 1LbF8RTlCAuSm_78kA7oLaj91MxlZqhro # public_y

Downloading...
From: https://drive.google.com/uc?id=1vHDhZSPmhithLVRRzkNw0ak_kk7PhInu
To: /workspace/project/V_trade.csv
100%|█████████████████████████████████████████| 230M/230M [00:01<00:00, 200MB/s]
Downloading...
From: https://drive.google.com/uc?id=145T8z3XXlsaISzWdrJgJVANGokG1XuFI
To: /workspace/project/V_remit.csv
100%|███████████████████████████████████████| 90.4M/90.4M [00:00<00:00, 219MB/s]
Downloading...
From: https://drive.google.com/uc?id=1AubrUmeNUgpgiOu4tay6Gwl8O3lBaokF
To: /workspace/project/V_info.csv
100%|████████████████████████████████████████| 557k/557k [00:00<00:00, 27.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zZo9RLt3mMmJZxEETSY2g9ND31qkZIn0
To: /workspace/project/V_cred.csv
100%|███████████████████████████████████████| 12.3M/12.3M [00:00<00:00, 115MB/s]
Downloading...
From: https://drive.google.com/uc?id=1uFCx21bqE3FnrdfvN_mwtw2-nvogTEex
To: /workspace/project/V_cons.csv
100%|█████████████████████████████████████████| 105M/105M [00:00<00:00, 20

In [404]:
V_cons = pd.read_csv('V_cons.csv').iloc[:, 1:]
V_cred = pd.read_csv('V_cred.csv').iloc[:, 1:]
V_info = pd.read_csv('V_info.csv').iloc[:, 1:]
V_remit = pd.read_csv('V_remit.csv').iloc[:, 1:]
V_trade = pd.read_csv('V_trade.csv').iloc[:, 1:]
train_y = pd.read_csv('train_y.csv').iloc[:, 1:]

V_cons_public = pd.read_csv('V_cons_public.csv').iloc[:, 1:]
V_cred_public = pd.read_csv('V_cred_public.csv').iloc[:, 1:]
V_info_public = pd.read_csv('V_info_public.csv').iloc[:, 1:]
V_remit_public = pd.read_csv('V_remit_public.csv').iloc[:, 1:]
V_trade_public = pd.read_csv('V_trade_public.csv').iloc[:, 1:]
public_y = pd.read_csv('public_y.csv').iloc[:, 1:]

all_keys = pd.read_csv('all_keys.csv')
ESun_public_y_answer = pd.read_csv('ESun_public_y_answer.csv')

In [405]:
V_overall = pd.concat([V_info, V_cred, V_cons, V_remit, V_trade], axis=1).fillna(0)
V_overall_public = pd.concat([V_info_public, V_cred_public, V_cons_public, V_remit_public, V_trade_public], axis=1).fillna(0)

In [406]:
print(V_info.shape)
print(V_cred.shape)
print(V_cons.shape)
print(V_remit.shape)
print(V_trade.shape)
print(V_overall.shape)
print()
print(V_info_public.shape)
print(V_cred_public.shape)
print(V_cons_public.shape)
print(V_remit_public.shape)
print(V_trade_public.shape)
print(V_overall_public.shape)
print()
print(train_y.shape)
print(public_y.shape)
print()
print(all_keys.shape)
print(ESun_public_y_answer.shape)

(23906, 4)
(23906, 117)
(23906, 1965)
(23906, 1572)
(23906, 3537)
(23906, 7195)

(1845, 4)
(1845, 117)
(1845, 1965)
(1845, 1572)
(1845, 3537)
(1845, 7195)

(23906, 1)
(1845, 1)

(25751, 1)
(1845, 2)


In [407]:
# instance = torch.tensor(V_overall.values).to(torch.float32)
# print(torch.any(torch.isnan(instance)))
# for row in range(len(instance)):
#     for i in instance[row]:
#         if torch.isnan(i):
#             print(row, instance[row])

In [408]:
print(public_y)
print(public_y.sum())

      0
0     0
1     0
2     0
3     0
4     0
...  ..
1840  0
1841  0
1842  0
1843  0
1844  0

[1845 rows x 1 columns]
0    11
dtype: int64


In [409]:
public_y

,0
0,0
1,0
2,0
3,0
4,0
...,...
1840,0
1841,0
1842,0
1843,0


# Main

In [440]:
# set parameters
n_epoch = 20
batch = 32 #
lr = 0.01
w = 1

In [441]:
# define network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.network = nn.Sequential( # 7195 -> 1
            nn.Linear(7195, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid() # last one must be sigmoid 
        )
                
    def forward(self, x):
        x = self.network(x)
        return x

In [442]:
def loss_function(prob, ans):
    #print(prob, ans)
    return (w * (1 - prob) * ans + (prob) * (1 - ans))

In [443]:
def train(train_data, val_data, model, n_epoch, batch, lr, device):
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    best_loss = 1000000
    for epoch in range(n_epoch):
        total_loss = 0
        model.train()
        idx = 0
        for data, ans in train_data:
            data, ans = data.to(device), ans.to(device)
            prob = model(data)
            loss = torch.sum(loss_function(prob, ans))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += (loss.item() / len(train_data))
            print('[Epoch %d | %d/%d] loss: %.4f' % ((epoch+1), idx*batch, len(train_data) * batch, loss.item()), end='\r')
            idx += 1
        print("\n  Training  | Loss:%.4f " % total_loss)

        # validation set
        model.eval()
        total_loss = 0
        idx = 0 
        with torch.no_grad():
            for data, ans in val_data:
                data, ans = data.to(device), ans.to(device)
                prob = model(data)
                loss = torch.sum(loss_function(prob, ans))
                total_loss += (loss.item() / len(val_data))
                idx += 1
            print(" Validation | Loss:%.4f " % total_loss)
        # save model
        if total_loss < best_loss:
                best_loss = total_loss
                print("saving model with loss %.4f...\n" % total_loss)
                torch.save(model.state_dict(), "%s" % "model.pth")
    return model

In [444]:
class TrainDataset(Dataset):
    def __init__(self):
        self.X = torch.tensor(V_overall.values).to(torch.float32)
        self.Y = torch.tensor(train_y.values).to(torch.float32)
  
    def __len__(self):
        return self.X.size(0)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

class ValDataset(Dataset):
    def __init__(self):
        self.X = torch.tensor(V_overall_public.values).to(torch.float32)
        self.Y = torch.tensor(public_y.values).to(torch.float32)

    def __len__(self):
        return self.X.size(0)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

In [445]:
trainset = TrainDataset()
valset = ValDataset()
train_dataloader = DataLoader(trainset, batch, True)
val_dataloader = DataLoader(valset, batch, False)

In [446]:
device = 'cuda:0'
model = Net().to(device) 
model = train(train_dataloader, val_dataloader, model, n_epoch, batch, lr, device)

[Epoch 1 | 23904/23936] loss: 0.0000
  Training  | Loss:0.3360 
 Validation | Loss:0.1897 
saving model with loss 0.1897...

[Epoch 2 | 23904/23936] loss: 0.0000
  Training  | Loss:0.3128 
 Validation | Loss:0.1897 
[Epoch 3 | 23904/23936] loss: 0.0000
  Training  | Loss:0.3128 
 Validation | Loss:0.1897 
[Epoch 4 | 23904/23936] loss: 0.0000
  Training  | Loss:0.3128 
 Validation | Loss:0.1897 
[Epoch 5 | 23904/23936] loss: 0.0000
  Training  | Loss:0.3128 
 Validation | Loss:0.1897 
[Epoch 6 | 23904/23936] loss: 0.0000
  Training  | Loss:0.3128 
 Validation | Loss:0.1897 
[Epoch 7 | 23904/23936] loss: 0.0000
  Training  | Loss:0.3128 
 Validation | Loss:0.1897 
[Epoch 8 | 23904/23936] loss: 0.0000
  Training  | Loss:0.3128 
 Validation | Loss:0.1897 
[Epoch 9 | 23904/23936] loss: 0.0000
  Training  | Loss:0.3128 
 Validation | Loss:0.1897 
[Epoch 10 | 23904/23936] loss: 0.0000
  Training  | Loss:0.3128 
 Validation | Loss:0.1897 
[Epoch 11 | 23904/23936] loss: 0.0000
  Training  | Los

# Result

In [447]:
best_model = model
best_model.load_state_dict(torch.load("model.pth"))
best_model = best_model.eval()

test_dataloader = DataLoader(valset, 1, False)
result = []
for x, _ in test_dataloader:
    x = x.to(device)
    result.append(best_model(x).item())
print(max(result))

0.0


In [448]:
keys_to_predict = sorted(ESun_public_y_answer['alert_key'].values.tolist())
pairs = np.array(list(zip(keys_to_predict, result)))
sorted_pairs = np.flip(pairs[pairs[:, 1].argsort()], 0)
print(sorted_pairs)

[[365073.      0.]
 [356630.      0.]
 [356634.      0.]
 ...
 [360600.      0.]
 [360601.      0.]
 [352249.      0.]]


In [449]:
index_list = []
SAR_count = 0
for key, flag in ESun_public_y_answer.values.tolist():
    if flag == 1:
        SAR_count += 1
        for idx in range(len(sorted_pairs)):
            if key == sorted_pairs[idx][0]:
                index_list.append(idx + 1)
                break
print(len(index_list))
print(SAR_count)
index_list.sort()
print(index_list[-2])
print("score: ", str((SAR_count - 1) / index_list[-2]))

# 0.01, 0.05, 0.1, 0.3

11
11
1384
score:  0.0072254335260115606
